## **Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Install dependencies**

In [6]:
!pip install roboflow

  Using cached roboflow-1.1.58-py3-none-any.whl.metadata (9.7 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
Using cached roboflow-1.1.58-py3-none-any.whl (84 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


## **Set locale to UTF-8**

In [7]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

## **Import Packages**

In [8]:
from roboflow import Roboflow

import cv2
import os

## **Downloading dataset from Roboflow**

In [9]:
rf = Roboflow(api_key="23YGtOSSXc7Mi257n5dt")

In [10]:
# BELOW ARE ALL FOR TRAINING CLASSIFICATION
project_traffic = rf.workspace("selfdriving-car-qtywx").project("self-driving-cars-lfjou")
version = project_traffic.version(6)
dataset = version.download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Self-Driving-Cars-6 in tensorflow:: 100%|██████████| 4977/4977 [00:01<00:00, 4870.54it/s]


In [11]:
project = rf.workspace("stage-aacaq").project("stage-tlvcc")
version = project.version(1)
dataset = version.download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Stage-1 in tensorflow:: 100%|██████████| 3412/3412 [00:00<00:00, 4060.31it/s]


In [12]:
project = rf.workspace("school-0ljld").project("kaggle-datasets-for-traffic")
version = project.version(2)
dataset = version.download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to kaggle-datasets-for-traffic-2 in tensorflow:: 100%|██████████| 4000/4000 [00:00<00:00, 6937.31it/s]


In [13]:
project = rf.workspace("trafic-object-detection-autonomous-no-augm").project("traffic-sign-detection-0y9yn")
version = project.version(2)
dataset = version.download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Traffic-sign-detection-2 in tensorflow:: 100%|██████████| 4770/4770 [00:00<00:00, 5113.29it/s]


## **Generate dataset for our own model from multiple datasets downloaded from Roboflow**

In [19]:
!rm -rf Dataset/

In [20]:
MIN_SIZE = 64 # filtering only width and height greater than 64

roboflow_data_paths = ["/content/Self-Driving-Cars-6",
                       "/content/kaggle-datasets-for-traffic-2",
                       "/content/Traffic-sign-detection-2"]

our_dataset_path = "/content/Dataset"
our_dataset_test_path = os.path.join(our_dataset_path, 'test')
our_dataset_train_path = os.path.join(our_dataset_path, 'train')
our_dataset_valid_path = os.path.join(our_dataset_path, 'valid')

os.makedirs(our_dataset_path, exist_ok=True)
os.makedirs(our_dataset_test_path, exist_ok=True)
os.makedirs(our_dataset_train_path, exist_ok=True)
os.makedirs(our_dataset_valid_path, exist_ok=True)

folders = ['test', 'train', 'valid']
target_labels = [ 'Speed Limit 10',
                  'Speed Limit 20',
                  'Speed Limit 30',
                  'Speed Limit 40',
                  'Speed Limit 50',
                  'Speed Limit 60',
                  'Speed Limit 70',
                  'Speed Limit 80',
                  'Speed Limit 90',
                  'Speed Limit 100',
                  'Speed Limit 110',
                  'Speed Limit 120',
                  'Stop']


for index, roboflow_data_path in enumerate(roboflow_data_paths):
  for data_type in folders:
    with open(os.path.join(roboflow_data_path, data_type, '_annotations.csv'), 'r') as fh:
      lines = fh.readlines()

      for line in lines:
        if line.strip() != '' and not line.startswith('filename'):
          splitted = line.strip().split(',')

          filename = splitted[0]
          label = splitted[3]
          if label not in target_labels:
            continue

          x1 = splitted[4]
          y1 = splitted[5]
          x2 = splitted[6]
          y2 = splitted[7]
          if int(x2) - int(x1) < MIN_SIZE or int(y2) - int(y1) < MIN_SIZE:
            continue

          # copy image
          img = cv2.imread(os.path.join(roboflow_data_path, data_type, filename))
          img = img[int(y1):int(y2), int(x1):int(x2)]
          cv2.imwrite(os.path.join(our_dataset_path, data_type, filename), img)
          # index indicates which dataset is it from;
          # some datasets have just way too small-sized traffic signs...
          # we are not going to use tiny-tinee traffic signs for creating adversarial samples.

          with open(os.path.join(our_dataset_path, data_type,'_annotations.csv'), "a") as file:
            file.write(line.strip() + ',' + str(index) + '\n')

    with open(os.path.join(our_dataset_path, data_type,'_annotations.csv'), "a") as file:
      file.write("\n")



## **BACKUP TO GOOGLE DRIVE**

In [21]:
!zip -r Dataset.zip Dataset && mv Dataset.zip /content/drive/MyDrive/ECE-653/

Streaming output truncated to the last 5000 lines.
  adding: Dataset/train/00007_00010_00023_png.rf.7835cbcfa6babb68c444a538b0062f17.jpg (deflated 1%)
  adding: Dataset/train/000010_jpg.rf.280eb2d57ed2f197721d34f6ca6c533e.jpg (deflated 2%)
  adding: Dataset/train/00000_00002_00010_png.rf.d57c8945142f6d675990162cdd0dec93.jpg (deflated 1%)
  adding: Dataset/train/000071_jpg.rf.8ebafdeafe82ad824de0313dfde2c888.jpg (deflated 4%)
  adding: Dataset/train/001466_jpg.rf.5f1e09ac149f516e67ac48f70c3b761d.jpg (deflated 1%)
  adding: Dataset/train/00007_00002_00010_png.rf.52a4637f1c61ee211104e9c6d1824d3b.jpg (deflated 1%)
  adding: Dataset/train/road426_png.rf.0cbcc9797ab58af5eefaeb13f28b0c30.jpg (deflated 4%)
  adding: Dataset/train/00000_00005_00002_png.rf.12521c166f3ca192f74ef5a18e5aa2cd.jpg (deflated 2%)
  adding: Dataset/train/001239_jpg.rf.26b4ca5e1f95a67cfe87a10160625899.jpg (deflated 2%)
  adding: Dataset/train/00008_00017_00023_png.rf.4e62fa0b1e9860ed7ef6fcb473637736.jpg (deflated 1%)
  a